In [34]:
from random import *
import numpy as nm
from scipy.stats import truncnorm

seed (None)

## Global variables
global N
global STATE0, STATE1
global BASE, OPT
global PDM, GREEDY
global iCache, dCache


## Instantiating global variables
N = 100000
## BASE list has all the base cycle values that were generated
## OPT list has the optimized values for that cycle post optimization

BASE = []
OPT = []
STATE0 = {'meanCycle': 50000, 'stdDev': 5000, 'min': 30000, 'max': 70000, 'base': BASE, 'opt': OPT}
STATE1 = {'meanCycle': 5000, 'stdDev': 500, 'min': 1000, 'max': 7000, 'base': BASE, 'opt': OPT}

#PDM = {'freqSampling': 2000, 'lat': 500, 'stdDev': 20, 'quality': 0.8}
#GREEDY = {'freqSampling': 4000, 'lat': 1500, 'stdDev': 100, 'quality': 0.9}

#iCache = {'lat': 250, 'stdDev': 10, 'quality': 0.2}
#dCache = {'lat': 400, 'stdDev': 50, 'quality': 0.3}

## name key added to print the name
PDM = {'name': 'PDM', 'freqSampling': 2000, 'lat': 500, 'stdDev': 20, 'quality': 0.8}
GREEDY = {'name': 'GREEDY', 'freqSampling': 4000, 'lat': 1500, 'stdDev': 100, 'quality': 0.9}

iCache = {'name': 'iCahche', 'lat': 250, 'stdDev': 10, 'quality': 0.2}
dCache = {'name': 'dCache', 'lat': 400, 'stdDev': 50, 'quality': 0.3}

def state_0():
    print "State_0"
    ## Lets take that the mean values of cycles in state0 is 50000 and std Dev is 5000
    global STATE0, BASE, OPT
    
    #cycle = STATE0['meanCycle']
    #stdDev = STATE0['stdDev']
    
    ## Creating a dictionary of list to keep track of our base cycles and optimized cycles
    ## base and opt are the lists corressponding to base and optimized cycles
    #base = []
    #opt = []
    
    #state0 = {'BASE': base, 'OPT': opt}
    
    ## Creating a base cycle and appending it to the list base
    
    ## baseCycles for the state at the current execution time
    ## random number from the normal distribution with mean=STATE1['meanCycle']
    ## and stdDev = STATE1['stdDev']
    #baseCycles = round(nm.random.normal(STATE0['meanCycle'], STATE0['stdDev']))
    baseCycles = round(getTruncatedNormal(STATE0['meanCycle'], STATE0['stdDev'],\
                                         STATE0['min'], STATE0['max']))
    BASE.append(baseCycles)
    
    ## For the first iteration we need to start with time=0
    if (len(BASE) == 1):
        optStateCycles = getOptCycles(baseCycles, 0, PDM, iCache)
    else:
        ## startCycles for next iteration is the baseCycle from previous state
        optStateCycles = getOptCycles(baseCycles, getTotalCycles(BASE, len(BASE)-1), PDM, iCache)
    
    OPT.append(optStateCycles)
    
    ## 
    #for i in range(len(BASE)):
        #print "BASE[%d] = %f" %(i, BASE[i])
    
    if random() < 0.9:
        return state_1
    else:
        return state_0

def state_1():
    print "State_1"
    
    ##
    global STATE1, BASE, OPT
    
    ## baseCycles for the state at the current execution time
    ## random number from the normal distribution with mean=STATE1['meanCycle']
    ## and stdDev = STATE1['stdDev']
    #baseCycles = round(nm.random.normal(STATE1['meanCycle'], STATE1['stdDev']))
    baseCycles = round(getTruncatedNormal(STATE1['meanCycle'], STATE1['stdDev'],\
                                         STATE1['min'], STATE1['max']))
    BASE.append(baseCycles)
    
    if (len(BASE) == 1):
        optStateCycles = getOptCycles(baseCycles, 0, PDM, iCache)
    else:
        optStateCycles = getOptCycles(baseCycles, getTotalCycles(BASE, len(BASE)-1), PDM, iCache)
    
    OPT.append(optStateCycles)
    
    #for i in range(len(BASE)):
        #print "BASE[%d]= %f" %(i, BASE[i])
        
    if random() > 0.3:
        return state_0
    else:
        return state_1
    
    
## Other functions

## sum() => internal function can also be used
## Usage sum(list)
def getTotalCycles(list1, index):
    tot = 0
    for i in range(index):
        tot = tot + list1[i]
    return tot

###############################################

## getOptCycles takes in input basecycle, time at which the state started,
## method and technique applied and returns the optimized time for that state, that method 
## and technique

## baseCycles => is generated randomly based on mean and variance for a state
## startCycles => is the time at which the previous state finished. Start of this state in globale
## time
## method => method, PDM, GREEDY, GENERIC
## method['lat'] => gives the latency time for the method
## tech => technique applied, iCache, dCache, ROB etc.

def getOptCycles(baseCycles, startCycles, method, tech):
    #print "method= %s\n%s['freqSampling'] = %d" %(method, method, method['freqSampling'])
    #print "Type of Method = %s" %(type(method))
    samplingRate = method['freqSampling']
    
    ## samplingTime is the time in the state where the technique will sample for optimisation
    ## If the startCyles is 0 => Starting the simulation
    ## startCycles is divisible by sampling rate => We discussed to go on to the next 
    ## sampling cycle
    if (startCycles % samplingRate == 0 or startCycles == 0):
        samplingTime = samplingRate
    elif (startCycles % samplingRate != 0):
        ## Since we know the starting time, we divide it by samplingRate to see how much 
        ## it is above the samplingRate. Now we subtract it from the samplingRate to get 
        ## how further down will the next sampling be done.
        ## We multiply 2*samplingRate since the program will take at least freqSampling
        ## to optimize.
        #samplingTime = startCycles + (samplingRate - nm.remainder(startCycles, samplingRate))
        samplingTime = (2*samplingRate) - nm.remainder(startCycles, samplingRate)
    
    if (samplingTime > baseCycles):
        print "Sampling Time greater than baseCycles ==> No optimisation possible\n"
        print "baseCycles= %d\nsamplingTime= %d\n" %(baseCycles, samplingTime)
        return baseCycles
    else:
        ## globalSamplingTime => Gives the sampling time with respect to global time
        globalSamplingTime = startCycles + samplingTime

        ## optTime => Represents the time taken by method to optimize
        optTime = round(nm.random.normal(method['lat'], method['stdDev']))

        ## configTime => Represents the time taken by technique to configure
        configTime = round(nm.random.normal(tech['lat'], tech['stdDev']))

        ## timeleft => Represents the time left after sampling and optimisation and configuration time
        timeLeft = baseCycles - (samplingTime + optTime + configTime)
        if (timeLeft < 0):
            print "Cannot optimize because there is no time left for optimisation\n"
            print "timeLeft < 0\n"
            print "baseCycles= %d\nstartCycles= %d\nsamplingTime= %d\nglobalSamplingTime=%d\noptTime= %d\
        \nconfigTime= %d\ntimeLeft= %d\n"\
        % (baseCycles, startCycles, samplingTime, globalSamplingTime, optTime, \
           configTime, timeLeft)
            return baseCycles
        
        else:
            ## this time is now optimized according to the method
            opt_timeLeft = round(method['quality']*timeLeft)

            newStateCycles = samplingTime + optTime + configTime + opt_timeLeft

            print "\nbaseCycles= %d\nstartCycles= %d\nsamplingTime= %d\nglobalSamplingTime=%d\noptTime= %d\
            \nconfigTime= %d\ntimeLeft= %d\nopt_timeLeft=%d\nnewStateCycles= %d\n"\
            % (baseCycles, startCycles, samplingTime, globalSamplingTime, optTime, \
               configTime, timeLeft, opt_timeLeft, newStateCycles)

            ## returning the optimized time the state takes because of the technique and method applied
            return newStateCycles

#######################################################    

## getTruncatedNormal gets random number by normal distribution within 'low' and 'up' given
## the mean and standard deviation of the numbers.
## rvs ==> is random variable continuous 
## Refer ==> https://stackoverflow.com/questions/36894191/how-to-get-a-normal-distribution-within-a-range-in-numpy

def getTruncatedNormal (mean, stdDev, low, up):
    return truncnorm ((low-mean)/stdDev, (up-mean)/stdDev, loc=mean, scale=stdDev).rvs()

#print "truncated Normal number between %d and %d with mean=%d and stdDev = %d is %d"\
#% (numMin, numMax, numMean, numSd, getTruncatedNormal(numMean, numSd, numMin, numMax).rvs() )    
################
    
state = state_0

count = 0

while state:
    state = state()
    ## when the sum of base goes above N we stop
    ## Stops close to the maximum number of cycles that we want to execute
    if sum(BASE) > N:
        ## state => None. while loop stops
        state = None


print "\ntype of BASE = %s\n" %(type(BASE))
#totalBaseCycles = getTotalCycles(BASE, len(BASE))

for i in range(len(BASE)):
    print "BASE[%d]= %d, OPT[%d]=%d" %(i, BASE[i], i, OPT[i])
    
#print "\nTotal Base Cycles = %d" %(totalBaseCycles)

### 
#getOptCycles(BASE[1], BASE[0], PDM, iCache)

print "\nTotal Base = %d, Total Opt = %d\nTotal Speed Up = %f" \
% (sum(BASE), sum(OPT), sum(BASE)/sum(OPT))


State_0

baseCycles= 52374
startCycles= 0
samplingTime= 2000
globalSamplingTime=2000
optTime= 469            
configTime= 250
timeLeft= 49655
opt_timeLeft=39724
newStateCycles= 42443

State_0

baseCycles= 45002
startCycles= 52374
samplingTime= 3626
globalSamplingTime=56000
optTime= 451            
configTime= 259
timeLeft= 40666
opt_timeLeft=32533
newStateCycles= 36869

State_1

baseCycles= 5421
startCycles= 97376
samplingTime= 2624
globalSamplingTime=100000
optTime= 523            
configTime= 260
timeLeft= 2014
opt_timeLeft=1611
newStateCycles= 5018


type of BASE = <type 'list'>

BASE[0]= 52374, OPT[0]=42443
BASE[1]= 45002, OPT[1]=36869
BASE[2]= 5421, OPT[2]=5018

Total Base = 102797, Total Opt = 84330
Total Speed Up = 1.218985


In [35]:
## Once BASE is created, we can use it to get speedup for 
## various METHODS and TECHNIQUES

## create OPT list for each Method
global PDM_OPT, GREEDY_OPT

## name key added to print the name
PDM = {'name': 'PDM', 'freqSampling': 2000, 'lat': 500, 'stdDev': 20, 'quality': 0.8}
GREEDY = {'name': 'GREEDY', 'freqSampling': 4000, 'lat': 1500, 'stdDev': 100, 'quality': 0.9}

iCache = {'name': 'iCahche', 'lat': 250, 'stdDev': 10, 'quality': 0.2}
dCache = {'name': 'dCache', 'lat': 400, 'stdDev': 50, 'quality': 0.3}

PDM_OPT = []
GREEDY_OPT = []

## Create a list of methods, do not create strings 'PDM' XX
METHOD = [PDM, GREEDY]
TECH = [iCache, dCache]

## Create METHOD_LIST in one to one coresspondence to METHOD List
METHOD_LIST = [PDM_OPT, GREEDY_OPT]

for i in range(len(TECH)):
    ## Instantiating optimizations lists to 0, for next iteration.
    ## Will have to do this for all the lists.
    ## Doing this does not help, python refers variables, hence 
    ## METHOD_LIST[j] = [] works to instantiate the lists.
    #PDM_OPT = []
    #GREEDY_OPT = []
    #print ("\ni= %d, len(PDM_OPT)= %d, len(GREEDY_OPT)= %d\n") %(i, len(PDM_OPT), len(GREEDY_OPT))
    for j in range(len(METHOD)):
        METHOD_LIST[j] = []
        for b in range(len(BASE)):
            #print "Ooptimisation Method = %s , Tech = %s" %(METHOD[j], TECH[i])
            #print "Type of method = %s type of PDM variable=%s\nb=%d type of b=%s" \
            #%(type(METHOD[j]), type(PDM), b, type(b))
            if (b==0):
                optCycle = getOptCycles(BASE[b], getTotalCycles(BASE, 0), METHOD[j], TECH[i])
            else:
                optCycle = getOptCycles(BASE[b], getTotalCycles(BASE, b), METHOD[j], TECH[i])
            
            optList = METHOD_LIST[j]
            #print "\nname= %s, optList len = %d\n" %(METHOD_LIST[j], len(optList))
            optList.append(optCycle)
        
        print "\n############################\n"
        print "Ooptimisation Method = %s , Tech = %s" %(METHOD[j]['name'], TECH[i]['name'])
        for k in range(len(BASE)):
            print "BASE[%d]= %d, optList[%d]=%d" %(k, BASE[k], k, optList[k])
        print "\nTotal Base = %d, Total Opt = %d\nTotal Speed Up = %f" \
        % (sum(BASE), sum(optList), sum(BASE)/sum(optList))
        print "\n############################\n"
        


baseCycles= 52374
startCycles= 0
samplingTime= 2000
globalSamplingTime=2000
optTime= 492            
configTime= 250
timeLeft= 49632
opt_timeLeft=39706
newStateCycles= 42448


baseCycles= 45002
startCycles= 52374
samplingTime= 3626
globalSamplingTime=56000
optTime= 515            
configTime= 236
timeLeft= 40625
opt_timeLeft=32500
newStateCycles= 36877


baseCycles= 5421
startCycles= 97376
samplingTime= 2624
globalSamplingTime=100000
optTime= 508            
configTime= 250
timeLeft= 2039
opt_timeLeft=1631
newStateCycles= 5013


############################

Ooptimisation Method = PDM , Tech = iCahche
BASE[0]= 52374, optList[0]=42448
BASE[1]= 45002, optList[1]=36877
BASE[2]= 5421, optList[2]=5013

Total Base = 102797, Total Opt = 84338
Total Speed Up = 1.218869

############################


baseCycles= 52374
startCycles= 0
samplingTime= 4000
globalSamplingTime=4000
optTime= 1542            
configTime= 267
timeLeft= 46565
opt_timeLeft=41909
newStateCycles= 47718


baseCycles= 45002


In [2]:
import numpy as nm
from scipy.stats import truncnorm

## Seed can be fixed to get replicable results
nm.random.seed(None)

## Here the mean is 20 and variance is 2
num = nm.random.normal(20, 2)
print "num = %f" %(num)

## Creating dictionary of lists

list1 = [1,2,3]
list2 = [4,5,6]
st0 = {'base': list1, 'opt': list2}
summ = sum(list1)
print "st0['base]= %s\nst0['opt]= %s" %(st0['base'], st0['opt'])
print "Sum of list1 = %d" %(summ)

print "length list1= %d, type= %s" %(len(list1), type(list1))

### rounding , rounds off to the int closest to it.
### 2.4 => 2 , 2.5 => 3 (in case of equal it rounds off to further int) 
print "Rounding these number"
print "2.09 = %d\n2.49 = %d\n2.51 = %d\n2.98 = %d" %(round(2.09), round(2.49), round(2.51), round(2.98))

## Remainder
print "\nRemiander = %d\n" %(nm.remainder(10,3))

## if else loop
a = 24
b = 4
if (a % b == 0 ):
    print "loop 1"
elif (a % b != 0):
    print "loop2"

## length of a list
list3 = []
print "\nlength of list3 = %d" %(len(list3))

## truncated normal distribution random number generation
numMin = 5
numMax = 100
numMean = 35
numSd = 20
print "Random Number between %d and %d is %s" %(numMin, numMax, truncnorm.rvs(numMin, numMax, size=1))

def getTruncatedNormal (mean, stdDev, low, up):
    return truncnorm ((low-mean)/stdDev, (up-mean)/stdDev, loc=mean, scale=stdDev).rvs()
print "truncated Normal number between %d and %d with mean=%d and stdDev = %d is %d"\
% (numMin, numMax, numMean, numSd, getTruncatedNormal(numMean, numSd, numMin, numMax) )

## max of a list 
bc = [1, 2, 13, 4]
print "max of bc = %d" %(max(bc))

num = 20.429250
st0['base]= [1, 2, 3]
st0['opt]= [4, 5, 6]
Sum of list1 = 6
length list1= 3, type= <type 'list'>
Rounding these number
2.09 = 2
2.49 = 2
2.51 = 3
2.98 = 3

Remiander = 1

loop 1

length of list3 = 0
Random Number between 5 and 100 is [ 5.12699724]
truncated Normal number between 5 and 100 with mean=35 and stdDev = 20 is 6
max of bc = 13
